# oak-d × spectacularAI × nerfstudio Google Colab ワークフロー

このノートブックは、oak-dで撮影したデータ（spectacularAI形式）をGoogle Colabにアップロードし、nerfstudioで学習・可視化・ダウンロードするまでの一連の手順をまとめたものです。

---

## 概要
1. **Colab環境セットアップ** - PyTorch(GPU対応)のインストールとGPU詳細確認
2. **データアップロード** - processed.zipをアップロード・展開・パス修正
3. **nerfstudioインストール** - nerfstudioのインストールと動作確認
4. **データ確認** - アップロードしたデータの詳細構造確認
5. **学習実行** - nerfstudioでの学習とGPU使用状況監視・進捗表示
6. **学習結果確認** - 学習済みモデル(.ckpt)とconfig.ymlの生成確認
7. **複数形式エクスポート** - 点群・カメラ・Gaussian Splatなど複数形式でエクスポート
8. **代替エクスポート** - エクスポートコマンドが失敗した場合の直接ファイルアクセス
9. **品質確認＆可視化** - 学習メトリクス・レンダリング結果・リソース使用状況の確認
10. **ダウンロード＆アーカイブ** - 各種3Dデータ形式のColab環境からのダウンロード

## 前提条件
- ローカルで `sai-cli process --format nerfstudio` により processed/ フォルダが作成済み
- processed.zip ファイルが準備済み

## 主な改善点
- **Windows互換性**: パス区切り文字の自動修正機能
- **GPU監視**: リアルタイムGPUメモリ使用量表示
- **エラーハンドリング**: 各段階での詳細なエラー診断
- **ファイル管理**: サイズ確認付きの選択的ダウンロード
- **進捗表示**: 経過時間とステップ数の監視
- **複数エクスポート**: pointcloud・cameras・gaussian-splatなど複数形式対応
- **ビューワ統合**: Colab内インタラクティブ3Dビューワ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. Google Colab環境セットアップ
import sys
print("=== Google Colab環境確認 ===")
print(f"Python version: {sys.version}")

# GPU確認
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

print("\n=== Google Colab固有設定 ===")
# Google Driveマウント（オプション）
# from google.colab import drive
# drive.mount('/content/drive')

# パッケージの最新化
!pip install --upgrade pip

print("\n=== GPU対応PyTorchインストール ===")
# Colab推奨のPyTorchインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# GPU利用可能性の詳細チェック
import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"CUDA version: {torch.version.cuda}")
    print("GPU学習環境が正常に設定されました")
else:
    print("GPU が利用できません")
    print("ランタイム → ランタイムのタイプを変更してください")
    print("注意: GPU なしでも学習は可能ですが、非常に時間がかかります")

=== Google Colab環境確認 ===
Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
name, memory.total [MiB], memory.free [MiB]
Tesla T4, 15360 MiB, 15095 MiB

=== Google Colab固有設定 ===
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2

=== GPU対応PyTorchインストール ===
Looking in indexes: https://download.pytorch.org/whl/cu118

CUDA available: True
GPU device: Tesla T4
GPU memory: 14.7 GB
CUDA version: 12.6
GPU学習環境が正常に設定されました


In [4]:
# 2. データアップロードとGoogle Colab最適化
print("=== Google Colab データアップロード ===")
print("processed.zip ファイルをアップロードしてください")
print("ファイルサイズが大きい場合は、Google Driveからの読み込みも可能です")

from google.colab import files
import zipfile
import os
import json

# ファイルアップロード
print("\n[方法1] 直接アップロード:")
uploaded = files.upload()

if not uploaded:
    print("\nファイルがアップロードされませんでした")
    print("\n[方法2] Google Driveを使用する場合:")
    print("以下のコードのコメントアウトを解除してください:")
    print("# from google.colab import drive")
    print("# drive.mount('/content/drive')")
    print("# import shutil")
    print("# shutil.copy('/content/drive/MyDrive/processed.zip', '/content/')")
else:
    print("アップロード完了")

# zipファイル展開処理
for filename in uploaded.keys() if uploaded else []:
    if filename.endswith('.zip'):
        print(f"\n{filename} を展開中...")
        try:
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('/content/drive/MyDrive/')
            print(f"{filename} を展開しました")
            os.remove(filename)  # zipファイルを削除してディスク容量節約
            print("zipファイルを削除しました（容量節約のため）")
        except Exception as e:
            print(f"展開エラー: {e}")
            continue

# Google Colab環境でのパス正規化
print("\n=== Colab環境用パス修正 ===")
processed_path = '/content/drive/MyDrive/processed'
if os.path.exists(processed_path):
    # transforms.jsonのパス区切り文字を修正
    transforms_path = os.path.join(processed_path, 'transforms.json')
    if os.path.exists(transforms_path):
        try:
            with open(transforms_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            path_fixed = False
            # Colab環境用にパスを正規化
            if 'frames' in data:
                for frame in data['frames']:
                    if 'file_path' in frame:
                        original_path = frame['file_path']
                        # バックスラッシュをスラッシュに変換
                        normalized_path = original_path.replace('\\', '/')
                        # 先頭が './' でない場合は追加
                        if not normalized_path.startswith('./'):
                            normalized_path = './' + normalized_path.lstrip('./')

                        if original_path != normalized_path:
                            frame['file_path'] = normalized_path
                            path_fixed = True

            if path_fixed:
                with open(transforms_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=2)
                print("transforms.json のパスをColab環境用に修正しました")
            else:
                print("transforms.json のパスは既に正規化されています")

        except Exception as e:
            print(f"パス修正エラー: {e}")
    else:
        print("transforms.json が見つかりません")

    # ディレクトリ構造確認
    print("\n=== アップロード結果確認 ===")
    !ls -la /content/drive/MyDrive/processed/

    # 画像フォルダ確認
    images_path = os.path.join(processed_path, 'images')
    if os.path.exists(images_path):
        image_count = len([f for f in os.listdir(images_path)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"画像ファイル数: {image_count}")

else:
    print("processed/ フォルダが見つかりません")
    print("zipファイルの内容を確認してください")

print("\nColab環境でのデータ準備が完了しました")

=== Google Colab データアップロード ===
processed.zip ファイルをアップロードしてください
ファイルサイズが大きい場合は、Google Driveからの読み込みも可能です

[方法1] 直接アップロード:


Saving processed.zip to processed.zip
アップロード完了

processed.zip を展開中...
processed.zip を展開しました
zipファイルを削除しました（容量節約のため）

=== Colab環境用パス修正 ===
transforms.json のパスをColab環境用に修正しました

=== アップロード結果確認 ===
total 3309
drwx------ 3 root root    4096 Aug 29 14:10 colmap
drwx------ 2 root root    4096 Aug 29 14:10 images
-rw------- 1 root root 3337406 Aug 29 14:10 sparse_pc.ply
-rw------- 1 root root   41824 Aug 29 14:10 transforms.json
画像ファイル数: 86

Colab環境でのデータ準備が完了しました


In [5]:
# 3. nerfstudio Google Colab専用インストール
print("=== nerfstudio Colab専用インストール ===")

# Colab環境でのnerfstudioインストール
print("nerfstudio をColab環境用にインストール中...")
print("これには数分かかる場合があります...")

# 依存関係とnerfstudioのインストール
!pip install nerfstudio

# インストール確認とバージョン表示
print("\n=== インストール確認 ===")
try:
    import nerfstudio
    print("nerfstudio が正常にインポートできました")

    # バージョン情報
    !pip show nerfstudio | grep Version

    # Colab環境での基本コマンド確認
    print("\n=== Colab環境でのnerfstudioコマンド確認 ===")
    !ns-train --help | head -10

    print("\n=== 利用可能なメソッド確認 ===")
    !ns-train -h | grep -A5 "method"

    print("\nnerfstudio のColab環境セットアップが完了しました")

except ImportError as e:
    print(f"nerfstudio インポートエラー: {e}")
    print("ランタイムを再起動してから再実行してください")
except Exception as e:
    print(f"セットアップエラー: {e}")

# Colab環境用の追加設定
print("\n=== Colab環境用最適化設定 ===")
import os
# 環境変数設定（Colab用）
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # GPU使用指定
os.environ['PYTHONPATH'] = '/content'      # Python パス設定

print("Colab環境用の最適化設定が完了しました")

=== nerfstudio Colab専用インストール ===
nerfstudio をColab環境用にインストール中...
これには数分かかる場合があります...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.8/580.8 kB 22.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 119.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 63.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 65.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 64.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.7/743.7 kB 38.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 185.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 192.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 111.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


=== インストール確認 ===
nerfstudio が正常にインポートできました
Version: 1.1.5

=== Colab環境でのnerfstudioコマンド確認 ===
Traceback (most recent call last):
  File "/usr/local/bin/ns-train", line 3, in <module>
    from nerfstudio.scripts.train import entrypoint
  File "/usr/local/lib/python3.12/dist-packages/nerfstudio/scripts/train.py", line 62, in <module>
    from nerfstudio.configs.method_configs import AnnotatedBaseConfigUnion
  File "/usr/local/lib/python3.12/dist-packages/nerfstudio/configs/method_configs.py", line 26, in <module>
    from nerfstudio.cameras.camera_optimizers import CameraOptimizerConfig
  File "/usr/local/lib/python3.12/dist-packages/nerfstudio/cameras/camera_optimizers.py", line 35, in <module>
    from nerfstudio.configs.base_config import InstantiateConfig
  File "/usr/local/lib/python3.12/dist-packages/nerfstudio/configs/base_config.py", line 24, in <module>
    from nerfstudio.utils import writer
  File "/usr/local/lib/python3.12/dist-packages/nerfstudio/utils/writer.py", line 31, i

In [1]:
# 4. Colab環境でのデータ検証
print("=== Google Colab データ検証 ===")

import os
import json

# Colab環境でのパス設定
processed_path = '/content/drive/MyDrive/processed'

# 基本構造確認
print("データ構造確認:")
if os.path.exists(processed_path):
    !find /content/drive/MyDrive/processed -type f | head -15
else:
    print("processed/ フォルダが見つかりません")
    print("セル2でのアップロードを確認してください")

# transforms.json 詳細確認
print("\n=== transforms.json 検証 ===")
transforms_path = os.path.join(processed_path, 'transforms.json')
if os.path.exists(transforms_path):
    print("transforms.json が見つかりました")

    try:
        with open(transforms_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 基本情報表示
        print(f"カメラ内部パラメータ確認:")
        if 'camera_angle_x' in data:
            print(f"  camera_angle_x: {data['camera_angle_x']}")
        if 'fl_x' in data:
            print(f"  focal length x: {data['fl_x']}")

        # フレーム情報確認
        if 'frames' in data:
            frame_count = len(data['frames'])
            print(f"  総フレーム数: {frame_count}")

            if frame_count > 0:
                sample_frame = data['frames'][0]
                sample_path = sample_frame.get('file_path', '')
                print(f"  サンプルパス: {sample_path}")

                # パス正規化確認
                if sample_path.startswith('./'):
                    print("  パス形式: 正規化済み (Colab対応)")
                else:
                    print("  警告: パス形式が正規化されていません")

                # 実際のファイル存在確認
                full_path = os.path.join(processed_path, sample_path.lstrip('./'))
                if os.path.exists(full_path):
                    print("  ファイル存在確認: OK")
                else:
                    print(f"  警告: ファイルが見つかりません: {full_path}")

        print("transforms.json の検証が完了しました")

    except Exception as e:
        print(f"transforms.json 読み込みエラー: {e}")
else:
    print("transforms.json が見つかりません")

# 画像データ確認
print("\n=== 画像データ検証 ===")
images_path = os.path.join(processed_path, 'images')
if os.path.exists(images_path):
    all_files = os.listdir(images_path)
    jpg_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg'))]
    png_files = [f for f in all_files if f.lower().endswith('.png')]

    total_images = len(jpg_files) + len(png_files)
    print(f"画像ファイル統計:")
    print(f"  JPG/JPEG: {len(jpg_files)} 枚")
    print(f"  PNG: {len(png_files)} 枚")
    print(f"  合計: {total_images} 枚")

    if total_images > 0:
        # サンプル画像の詳細確認
        sample_image = jpg_files[0] if jpg_files else png_files[0]
        sample_path = os.path.join(images_path, sample_image)
        file_size = os.path.getsize(sample_path) / 1024  # KB
        print(f"  サンプル画像: {sample_image} ({file_size:.1f} KB)")

        # Colab学習推奨設定の判定
        if total_images >= 20:
            print("Colab学習に適した画像数です")
        elif total_images >= 10:
            print("学習可能ですが、より多くの画像があると品質が向上します")
        else:
            print("警告: 画像数が少なく、学習結果の品質が低い可能性があります")

        print(f"\nColab環境でのデータ検証完了")
        print(f"学習準備が整いました！")
    else:
        print("画像ファイルが見つかりません")
else:
    print("images/ フォルダが見つかりません")

# その他のファイル確認
print("\n=== オプションファイル確認 ===")
additional_files = [
    ('sparse_pc.ply', 'スパース点群データ'),
    ('colmap', 'COLMAPデータ')
]

for file_name, description in additional_files:
    file_path = os.path.join(processed_path, file_name)
    if os.path.exists(file_path):
        print(f"  {description}: 利用可能")
    else:
        print(f"  {description}: なし（オプション）")

=== Google Colab データ検証 ===
データ構造確認:
/content/drive/MyDrive/processed/colmap/sparse/0/cameras.txt
/content/drive/MyDrive/processed/colmap/sparse/0/images.txt
/content/drive/MyDrive/processed/colmap/sparse/0/points3D.txt
/content/drive/MyDrive/processed/images/depth_00001.png
/content/drive/MyDrive/processed/images/depth_00002.png
/content/drive/MyDrive/processed/images/depth_00003.png
/content/drive/MyDrive/processed/images/depth_00004.png
/content/drive/MyDrive/processed/images/depth_00005.png
/content/drive/MyDrive/processed/images/depth_00006.png
/content/drive/MyDrive/processed/images/depth_00007.png
/content/drive/MyDrive/processed/images/depth_00008.png
/content/drive/MyDrive/processed/images/depth_00009.png
/content/drive/MyDrive/processed/images/depth_00010.png
/content/drive/MyDrive/processed/images/depth_00011.png
/content/drive/MyDrive/processed/images/depth_00012.png

=== transforms.json 検証 ===
transforms.json が見つかりました
カメラ内部パラメータ確認:
  focal length x: 426.952
  総フレーム数: 43
  サ

In [ ]:
# 5. Google Colab最適化 nerfstudio学習
print("=== Google Colab 最適化学習実行 ===")

import subprocess
import sys
import os
import time
import threading
import torch
import glob

# Colab環境確認
print("Colab学習環境確認:")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  GPU: {gpu_name}")
    print(f"  VRAM: {gpu_memory:.1f} GB")

    # GPU種別に応じた推奨設定
    if "T4" in gpu_name:
        recommended_iterations = 10000
        print(f"  T4 GPU検出: 軽量設定を推奨")
    elif "A100" in gpu_name or "V100" in gpu_name:
        recommended_iterations = 30000
        print(f"  高性能GPU検出: 高品質設定が可能")
    else:
        recommended_iterations = 15000
        print(f"  標準設定を使用")
else:
    recommended_iterations = 5000
    print("  CPU学習: 最小設定（非推奨）")

print(f"\n推奨イテレーション数: {recommended_iterations}")
print("学習時間は画像数とGPUにより5分〜60分程度です")
print("Ctrl+C で安全に中断できます（途中保存あり）\n")

# Colab固有のGPU監視機能
def monitor_colab_resources():
    """Colab環境のリソース監視"""
    if torch.cuda.is_available():
        while True:
            try:
                # GPU メモリ使用量
                memory_used = torch.cuda.memory_allocated(0) / 1024**3
                memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
                memory_percent = (memory_used / memory_total) * 100

                # CPU とメモリ（簡易表示）
                print(f"GPU: {memory_used:.1f}/{memory_total:.1f}GB ({memory_percent:.1f}%)", end='\r')
                time.sleep(30)
            except:
                break

# Colab最適化された学習コマンド
cmd = [
    "ns-train",
    "nerfacto",  # Colab環境で最も安定したメソッド
    "--data", "/content/drive/MyDrive/processed",
    "--output-dir", "/content/drive/MyDrive/processed/outputs/",
    "--max-num-iterations", str(recommended_iterations),
    "--steps-per-save", "1000",  # Colab環境では頻繁に保存
    "--steps-per-eval-image", "500",  # 評価頻度
    "--steps-per-eval-batch", "500",  # バッチ評価
    "--logging.steps-per-log", "100"  # ログ頻度（Colab表示用）
]

# GPU メモリに応じたバッチサイズ調整
if torch.cuda.is_available():
    memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    if memory_gb < 12:  # T4など
        cmd.extend(["--pipeline.datamanager.train-num-rays-per-batch", "2048"])
        print("GPU メモリ使用量を最適化しました（小メモリ GPU 対応）")
    elif memory_gb >= 24:  # A100など
        cmd.extend(["--pipeline.datamanager.train-num-rays-per-batch", "8192"])
        print("高メモリ GPU 用に最適化しました")

print(f"実行コマンド: {' '.join(cmd)}\n")

# リソース監視開始
if torch.cuda.is_available():
    monitor_thread = threading.Thread(target=monitor_colab_resources, daemon=True)
    monitor_thread.start()

# 学習実行
start_time = time.time()
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1,
    cwd="/content/drive/MyDrive/processed"  # Colab作業ディレクトリ
)

try:
    line_count = 0
    for line in process.stdout:
        print(line, end="")
        sys.stdout.flush()
        line_count += 1

        # Colab用の進捗表示（200行ごと）
        if line_count % 200 == 0:
            elapsed = time.time() - start_time
            print(f"\n[Colab進捗] 経過時間: {elapsed/60:.1f}分, 処理行数: {line_count}")

except KeyboardInterrupt:
    print("\n\n学習を中断しました（Colab環境）")
    process.terminate()
    process.wait()

    # 中断時の結果確認
    print("中断時点での学習結果を確認中...")
    if os.path.exists("/content/drive/MyDrive/processed/outputs"):
        ckpt_files = glob.glob('/content/drive/MyDrive/processed/outputs/**/*.ckpt', recursive=True)
        if ckpt_files:
            print(f"保存されたチェックポイント: {len(ckpt_files)} 個")
            latest_ckpt = max(ckpt_files, key=os.path.getctime)
            print(f"最新チェックポイント: {latest_ckpt}")
        else:
            print("チェックポイントファイルが見つかりません")

# 学習完了処理
process.wait()
elapsed_total = time.time() - start_time

print(f"\n=== Google Colab学習完了 ===")
print(f"総学習時間: {elapsed_total/60:.1f}分")
print(f"出力先: /content/drive/MyDrive/processed/outputs/")

# Colab環境での結果確認
if os.path.exists("/content/drive/MyDrive/processed/outputs"):
    ckpt_files = glob.glob('/content/drive/MyDrive/processed/outputs/**/*.ckpt', recursive=True)
    config_files = glob.glob('/content/drive/MyDrive/processed/outputs/**/config.yml', recursive=True)

    print(f"生成ファイル:")
    print(f"  チェックポイント (.ckpt): {len(ckpt_files)} 個")
    print(f"  設定ファイル (config.yml): {len(config_files)} 個")

    if ckpt_files and config_files:
        print("\nColab環境での学習が正常に完了しました！")
        print("次のセルでエクスポート処理を実行できます")
    else:
        print("\n学習結果が不完全です")
        print("ランタイムエラーがないか確認してください")
else:
    print("outputs/ フォルダが作成されませんでした")
    print("学習コマンドの実行に問題があった可能性があります")

=== Google Colab 最適化学習実行 ===
Colab学習環境確認:
  GPU: Tesla T4
  VRAM: 14.7 GB
  T4 GPU検出: 軽量設定を推奨

推奨イテレーション数: 10000
学習時間は画像数とGPUにより5分〜60分程度です
Ctrl+C で安全に中断できます（途中保存あり）

実行コマンド: ns-train nerfacto --data /content/drive/MyDrive/processed --output-dir /content/drive/MyDrive/processed/outputs/ --max-num-iterations 10000 --steps-per-save 1000 --steps-per-eval-image 500 --steps-per-eval-batch 500 --logging.steps-per-log 100

2025-08-29 14:22:45.089541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756477365.111421    6477 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756477365.119458    6477 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:175647

In [ ]:
# 6. 学習結果確認
print("=== 学習結果確認 ===")

import os
import glob

# 出力フォルダの存在確認
if os.path.exists("/content/drive/MyDrive/processed/outputs"):
    print("/content/drive/MyDrive/processed/outputs/ フォルダが見つかりました")

    # チェックポイントファイル確認
    ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
    config_files = glob.glob('outputs/**/config.yml', recursive=True)

    print(f"チェックポイントファイル: {len(ckpt_files)} 個")
    if ckpt_files:
        for ckpt in ckpt_files:
            size_mb = os.path.getsize(ckpt) / (1024 * 1024)
            print(f"  - {ckpt} ({size_mb:.1f} MB)")

    print(f"設定ファイル: {len(config_files)} 個")
    if config_files:
        for config in config_files:
            print(f"  - {config}")

    # フォルダ構造確認
    print("\n=== outputs/ フォルダ構造 ===")
    for root, dirs, files in os.walk("/content/drive/MyDrive/processed/outputs"):
        level = root.replace("/content/drive/MyDrive/processed/outputs", "").count(os.sep)
        indent = " " * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = " " * 2 * (level + 1)
        for file in files[:3]:  # 最初の3ファイルのみ表示
            print(f"{subindent}{file}")
        if len(files) > 3:
            print(f"{subindent}... 他 {len(files)-3} ファイル")

    if ckpt_files and config_files:
        print("\n学習完了！エクスポート処理に進めます")
    else:
        print("\n学習結果が不完全です。セル5を再実行してください")

else:
    print("outputs/ フォルダが見つかりません")
    print("セル5で学習を実行してください")

In [ ]:
# 7. Google Colab最適化エクスポート
print("=== Google Colab 最適化エクスポート ===")
print("Colab環境で安定動作する3形式でエクスポートします")

import glob
import os
import subprocess

# Colab環境での設定ファイル検索
config_files = glob.glob('/content/drive/MyDrive/processed/outputs/**/config.yml', recursive=True)

if not config_files:
    print("config.yml ファイルが見つかりません")
    print("セル5で学習を完了してから実行してください")
else:
    config_path = config_files[0]
    print(f"設定ファイル: {config_path}")

    # Colab環境用エクスポートディレクトリ
    export_dir = "/content/drive/MyDrive/processed/exports"
    os.makedirs(export_dir, exist_ok=True)

    # Colab環境で動作確認済みのエクスポート形式
    export_options = [
        ("pointcloud", "3D点群", "MeshLab等で表示可能な.plyファイル"),
        ("cameras", "カメラ軌道", "撮影経路を記録した.jsonファイル"),
        ("splat", "3D Gaussian", "高品質3D表現データ")
    ]

    successful_exports = []
    print(f"\nColab環境での{len(export_options)}形式エクスポート開始...")

    for option, name, description in export_options:
        output_path = os.path.join(export_dir, option)
        os.makedirs(output_path, exist_ok=True)

        print(f"\n{name}をエクスポート中...")
        print(f"  形式: {description}")

        # Colab環境用エクスポートコマンド
        cmd = [
            "ns-export", option,
            "--load-config", config_path,
            "--output-dir", output_path
        ]

        # Colab環境用の環境変数設定
        export_env = os.environ.copy()
        export_env['PYTHONWARNINGS'] = 'ignore'
        export_env['TORCH_WARN'] = '0'
        export_env['CUDA_VISIBLE_DEVICES'] = '0'  # Colab GPU指定

        try:
            print(f"  実行中: {' '.join(cmd)}")
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=600,  # Colab環境では10分でタイムアウト
                env=export_env,
                cwd="/content/drive/MyDrive/processed"
            )

            # エクスポート結果確認
            if os.path.exists(output_path):
                generated_files = [f for f in os.listdir(output_path)
                                 if os.path.isfile(os.path.join(output_path, f))]

                if generated_files:
                    print(f"  ✓ {name}エクスポート成功！")
                    successful_exports.append((option, name, output_path))

                    # ファイル詳細表示
                    total_size = 0
                    for i, file in enumerate(generated_files[:3]):  # 最初の3ファイル表示
                        file_path = os.path.join(output_path, file)
                        file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
                        total_size += file_size
                        print(f"    - {file} ({file_size:.2f} MB)")

                    if len(generated_files) > 3:
                        remaining_files = generated_files[3:]
                        remaining_size = sum(os.path.getsize(os.path.join(output_path, f))
                                           for f in remaining_files) / (1024 * 1024)
                        total_size += remaining_size
                        print(f"    - ...他 {len(remaining_files)} ファイル ({remaining_size:.2f} MB)")

                    print(f"    合計: {len(generated_files)} ファイル, {total_size:.2f} MB")
                else:
                    print(f"  ✗ {name}: ファイル生成に失敗")
                    if result.stderr:
                        # Colabで重要なエラーのみ表示
                        error_lines = [line for line in result.stderr.split('\n')
                                     if 'error' in line.lower() or 'exception' in line.lower()]
                        for error_line in error_lines[:2]:  # 最初の2行のみ
                            print(f"    エラー: {error_line.strip()}")

        except subprocess.TimeoutExpired:
            print(f"  ✗ {name}エクスポートがタイムアウトしました（10分）")
            print(f"    Colab環境では処理時間に制限があります")
        except Exception as e:
            print(f"  ✗ {name}エクスポート処理でエラー: {e}")

    # Colab環境での結果サマリー
    print(f"\n=== Colabエクスポート結果 ===")
    if successful_exports:
        print(f"成功: {len(successful_exports)}/{len(export_options)} 形式")
        print(f"エクスポート先: {export_dir}")

        for option, name, path in successful_exports:
            file_count = len([f for f in os.listdir(path)
                             if os.path.isfile(os.path.join(path, f))])
            folder_size = sum(os.path.getsize(os.path.join(path, f))
                             for f in os.listdir(path)
                             if os.path.isfile(os.path.join(path, f))) / (1024 * 1024)
            print(f"  ✓ {name}: {file_count} ファイル ({folder_size:.1f} MB)")

        print(f"\n次のステップ:")
        print(f"  - セル9: ファイルダウンロード")
        print(f"  - セル10: Colab内3Dビューワ")
    else:
        print("すべてのエクスポートが失敗しました")
        print("セル8の代替エクスポート方法をお試しください")

print(f"\nColab環境でのエクスポート処理が完了しました")

In [ ]:
# 8. Google Colab代替エクスポート（セーフモード）
print("=== Google Colab 代替エクスポート ===")
print("エクスポートコマンドが失敗した場合のColab用直接ファイルアクセス")

import glob
import os
import shutil
import json
from datetime import datetime

# Colab環境でのoutputsディレクトリ確認
outputs_dir = '/content/drive/MyDrive/processed/outputs'
if os.path.exists(outputs_dir):
    # Colab用代替エクスポートディレクトリ
    alt_export_dir = "/content/drive/MyDrive/processed/alternative_exports"
    os.makedirs(alt_export_dir, exist_ok=True)

    print(f"Colab環境: {outputs_dir} から重要ファイルを抽出中...")

    # 1. チェックポイントファイル収集
    ckpt_files = glob.glob(f'{outputs_dir}/**/*.ckpt', recursive=True)
    if ckpt_files:
        ckpt_dir = os.path.join(alt_export_dir, "checkpoints")
        os.makedirs(ckpt_dir, exist_ok=True)

        print(f"\nチェックポイントファイル処理中...")
        for ckpt_file in ckpt_files:
            dest_file = os.path.join(ckpt_dir, os.path.basename(ckpt_file))
            shutil.copy2(ckpt_file, dest_file)
            size_mb = os.path.getsize(dest_file) / (1024 * 1024)
            print(f"  ✓ {os.path.basename(ckpt_file)} ({size_mb:.1f} MB)")

    # 2. 設定ファイル収集
    config_files = glob.glob(f'{outputs_dir}/**/config.yml', recursive=True)
    if config_files:
        config_dir = os.path.join(alt_export_dir, "configs")
        os.makedirs(config_dir, exist_ok=True)

        print(f"\n設定ファイル処理中...")
        for i, config_file in enumerate(config_files):
            dest_name = f"config_{i+1}.yml"
            dest_file = os.path.join(config_dir, dest_name)
            shutil.copy2(config_file, dest_file)
            print(f"  ✓ {dest_name}")

    # 3. 学習ログとメトリクス収集
    log_extensions = ['*.json', '*.txt', '*.csv']
    log_files = []
    for ext in log_extensions:
        log_files.extend(glob.glob(f'{outputs_dir}/**/{ext}', recursive=True))

    if log_files:
        logs_dir = os.path.join(alt_export_dir, "logs")
        os.makedirs(logs_dir, exist_ok=True)

        print(f"\nログファイル処理中...")
        colab_size_limit = 50 * 1024 * 1024  # Colab環境では50MB制限
        total_size = 0

        for log_file in log_files[:10]:  # Colab環境では最大10ファイル
            file_size = os.path.getsize(log_file)
            if total_size + file_size < colab_size_limit:
                dest_file = os.path.join(logs_dir, os.path.basename(log_file))
                shutil.copy2(log_file, dest_file)
                size_mb = file_size / (1024 * 1024)
                total_size += file_size
                print(f"  ✓ {os.path.basename(log_file)} ({size_mb:.2f} MB)")
            else:
                print(f"  - サイズ制限によりスキップ: {os.path.basename(log_file)}")
                break

    # 4. Colab用メタデータ作成
    colab_metadata = {
        "export_method": "colab_alternative_direct_copy",
        "timestamp": datetime.now().isoformat(),
        "colab_environment": True,
        "source_directory": outputs_dir,
        "checkpoint_files": len(ckpt_files) if ckpt_files else 0,
        "config_files": len(config_files) if config_files else 0,
        "log_files": len([f for f in os.listdir(logs_dir)]) if os.path.exists(logs_dir) else 0,
        "total_size_mb": sum(os.path.getsize(os.path.join(root, file))
                            for root, dirs, files in os.walk(alt_export_dir)
                            for file in files) / (1024 * 1024),
        "notes": "Exported in Google Colab environment using direct file access"
    }

    metadata_file = os.path.join(alt_export_dir, "colab_export_info.json")
    with open(metadata_file, 'w', encoding='utf-8') as f:
        json.dump(colab_metadata, f, indent=2)

    # 5. Colab環境用ディレクトリ構造表示
    print(f"\n=== Colab代替エクスポート完了 ===")
    print(f"エクスポート先: {alt_export_dir}")
    print(f"総サイズ: {colab_metadata['total_size_mb']:.1f} MB")

    print(f"\nフォルダ構造:")
    for root, dirs, files in os.walk(alt_export_dir):
        level = root.replace(alt_export_dir, '').count(os.sep)
        indent = '  ' * level
        folder_name = os.path.basename(root) if root != alt_export_dir else 'alternative_exports'
        print(f"{indent}{folder_name}/")

        subindent = '  ' * (level + 1)
        for file in files[:5]:  # 各フォルダ最大5ファイル表示
            print(f"{subindent}- {file}")
        if len(files) > 5:
            print(f"{subindent}- ...他 {len(files)-5} ファイル")

    print(f"\nColab環境での使用方法:")
    print(f"  - checkpoints/: ローカル環境でのnerfstudio読み込み用")
    print(f"  - configs/: 学習設定確認・再現用")
    print(f"  - logs/: 学習進捗分析・可視化用")
    print(f"  - colab_export_info.json: Colab環境情報")

    print(f"\nファイルサイズがColab制限内に最適化されました")

else:
    print("outputs/ フォルダが見つかりません")
    print("セル5でのColab学習を完了してから実行してください")

print(f"\nColab代替エクスポート処理が完了しました")

In [ ]:
# 10. Google Colab品質確認＆可視化
print("=== Google Colab 学習結果確認 ===")

import os
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time

# Colab環境での結果ディレクトリ確認
outputs_dir = '/content/drive/MyDrive/processed/outputs'

if os.path.exists(outputs_dir):
    print(f"Colab環境: {outputs_dir} の学習結果を確認中...")

    # 1. Colab学習メトリクスの確認
    experiment_dirs = [d for d in os.listdir(outputs_dir)
                      if os.path.isdir(os.path.join(outputs_dir, d))]

    if experiment_dirs:
        latest_experiment = sorted(experiment_dirs)[-1]
        exp_dir = os.path.join(outputs_dir, latest_experiment)
        print(f"\n最新実験ディレクトリ: {latest_experiment}")

        # 2. Colab学習統計情報
        config_file = os.path.join(exp_dir, 'config.yml')
        if os.path.exists(config_file):
            print(f"\n=== Colab学習設定 ===")
            with open(config_file, 'r') as f:
                config_content = f.read()[:1000]  # Colab表示用に制限
                print(config_content[:500] + "..." if len(config_content) > 500 else config_content)

        # 3. Colabメトリクス可視化
        metrics_files = glob.glob(os.path.join(exp_dir, '**/*metrics*.json'), recursive=True)
        if metrics_files:
            print(f"\n=== Colab学習メトリクス可視化 ===")

            # Colab用サンプルメトリクス可視化
            plt.figure(figsize=(12, 4))

            try:
                with open(metrics_files[0], 'r') as f:
                    for i, line in enumerate(f):
                        if i > 100:  # Colab環境では100行制限
                            break
                        data = json.loads(line)
                        if 'step' in data and 'loss' in data:
                            plt.subplot(1, 3, 1)
                            plt.plot(data['step'], data['loss'], 'b.', alpha=0.5)
                            plt.title('Training Loss (Colab)')
                            plt.xlabel('Step')
                            plt.ylabel('Loss')

                            if 'psnr' in data:
                                plt.subplot(1, 3, 2)
                                plt.plot(data['step'], data['psnr'], 'g.', alpha=0.5)
                                plt.title('PSNR (Colab)')
                                plt.xlabel('Step')
                                plt.ylabel('PSNR (dB)')

                            if 'learning_rate' in data:
                                plt.subplot(1, 3, 3)
                                plt.plot(data['step'], data['learning_rate'], 'r.', alpha=0.5)
                                plt.title('Learning Rate (Colab)')
                                plt.xlabel('Step')
                                plt.ylabel('LR')

                plt.tight_layout()
                plt.savefig('/content/drive/MyDrive/processed/colab_training_metrics.png', dpi=100, bbox_inches='tight')
                plt.show()
                print("Colab学習メトリクスの可視化が完了しました")

            except Exception as e:
                print(f"メトリクス読み込みエラー (Colab互換性): {e}")

        # 4. Colab環境レンダリング画像確認
        render_dirs = glob.glob(os.path.join(exp_dir, '**/renders'), recursive=True)
        if render_dirs:
            render_dir = render_dirs[0]
            render_images = glob.glob(os.path.join(render_dir, '*.png'))[:6]  # Colab用6枚制限

            if render_images:
                print(f"\n=== Colab レンダリング結果確認 ===")
                print(f"レンダリング画像数: {len(render_images)}")

                # Colab用画像グリッド表示
                fig, axes = plt.subplots(2, 3, figsize=(12, 8))
                axes = axes.flatten()

                for i, img_path in enumerate(render_images):
                    if i >= 6:  # Colab表示制限
                        break
                    try:
                        img = Image.open(img_path)
                        # Colab用リサイズ（メモリ節約）
                        img.thumbnail((400, 400), Image.Resampling.LANCZOS)

                        axes[i].imshow(img)
                        axes[i].set_title(f'Render {i+1} (Colab)', fontsize=10)
                        axes[i].axis('off')
                    except Exception as e:
                        axes[i].text(0.5, 0.5, f'Load Error\n{e}',
                                   ha='center', va='center', transform=axes[i].transAxes)
                        axes[i].axis('off')

                # 空のサブプロットを非表示
                for i in range(len(render_images), 6):
                    axes[i].axis('off')

                plt.tight_layout()
                plt.savefig('/content/drive/MyDrive/processed/colab_render_samples.png', dpi=100, bbox_inches='tight')
                plt.show()
                print("Colab環境でのレンダリング結果表示が完了しました")

        # 5. Colab環境ファイルサイズ確認
        print(f"\n=== Colab環境ファイルサイズ確認 ===")
        total_size = 0
        file_types = {}

        for root, dirs, files in os.walk(exp_dir):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    size = os.path.getsize(file_path)
                    total_size += size

                    ext = os.path.splitext(file)[1].lower()
                    if ext in file_types:
                        file_types[ext] += size
                    else:
                        file_types[ext] = size
                except:
                    continue

        print(f"合計サイズ: {total_size / (1024**3):.2f} GB")
        print(f"ファイル種別サイズ:")
        for ext, size in sorted(file_types.items(), key=lambda x: x[1], reverse=True)[:10]:
            size_mb = size / (1024**2)
            if size_mb > 1:
                print(f"  {ext if ext else 'no_ext'}: {size_mb:.1f} MB")

        # 6. Colab環境リソース使用状況
        print(f"\n=== Colab リソース使用状況 ===")

        # GPU使用状況 (Colab環境)
        try:
            import torch
            if torch.cuda.is_available():
                gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
                gpu_allocated = torch.cuda.memory_allocated(0) / (1024**3)
                gpu_cached = torch.cuda.memory_reserved(0) / (1024**3)

                print(f"GPU総メモリ: {gpu_memory:.1f} GB")
                print(f"GPU使用中: {gpu_allocated:.1f} GB ({gpu_allocated/gpu_memory*100:.1f}%)")
                print(f"GPU予約済み: {gpu_cached:.1f} GB ({gpu_cached/gpu_memory*100:.1f}%)")
        except:
            print("GPU情報の取得に失敗しました")

        # RAM使用状況 (Colab環境)
        try:
            import psutil
            ram = psutil.virtual_memory()
            print(f"RAM総容量: {ram.total / (1024**3):.1f} GB")
            print(f"RAM使用中: {ram.used / (1024**3):.1f} GB ({ram.percent:.1f}%)")
            print(f"RAM利用可能: {ram.available / (1024**3):.1f} GB")
        except:
            print("RAM情報の取得に失敗しました")

        print(f"\n✓ Google Colab環境での品質確認が完了しました")
        print(f"画像は /content/drive/MyDrive/processed/ に保存されました（ダウンロード可能）")

    else:
        print("実験ディレクトリが見つかりません")
        print("セル5でのColab学習を完了してから実行してください")

else:
    print("outputs/ フォルダが見つかりません")
    print("セル5でのColab学習を完了してから実行してください")

In [ ]:
# 11. Google Colab ダウンロード＆アーカイブ作成
print("=== Google Colab ダウンロード用ファイル準備 ===")

import os
import zipfile
import shutil
import json
from datetime import datetime
from google.colab import files

# Colab環境確認
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("Google Colab環境を検出しました")

    # 1. Colab用ダウンロードディレクトリ作成
    download_dir = "/content/drive/MyDrive/processed/colab_downloads"
    os.makedirs(download_dir, exist_ok=True)

    # 2. 各種エクスポートファイルを収集
    files_to_download = []

    # outputs/からの学習結果
    outputs_dir = '/content/drive/MyDrive/processed/outputs'
    if os.path.exists(outputs_dir):
        print("\n学習結果ファイルを収集中...")

        # 最新実験ディレクトリを特定
        experiment_dirs = [d for d in os.listdir(outputs_dir)
                          if os.path.isdir(os.path.join(outputs_dir, d))]

        if experiment_dirs:
            latest_exp = sorted(experiment_dirs)[-1]
            exp_dir = os.path.join(outputs_dir, latest_exp)

            # 重要ファイルのみを選択的に収集（Colabサイズ制限対応）
            important_files = [
                ('config.yml', 'Config'),
                ('*.ckpt', 'Checkpoint'),
                ('**/cameras.json', 'Cameras'),
                ('**/dataparser_transforms.json', 'Transforms'),
                ('**/point_cloud.ply', 'PointCloud'),
                ('**/*.splat', 'GaussianSplat')
            ]

            for pattern, file_type in important_files:
                import glob
                matches = glob.glob(os.path.join(exp_dir, pattern), recursive=True)
                for match in matches[:3]:  # Colab用に各種類3ファイルまで
                    if os.path.getsize(match) < 100 * 1024 * 1024:  # 100MB制限
                        files_to_download.append((match, file_type))

    # alternative_exportsからのファイル
    alt_export_dir = "/content/drive/MyDrive/processed/alternative_exports"
    if os.path.exists(alt_export_dir):
        print("代替エクスポートファイルを収集中...")
        for root, dirs, files in os.walk(alt_export_dir):
            for file in files[:10]:  # Colab用制限
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) < 50 * 1024 * 1024:  # 50MB制限
                    files_to_download.append((file_path, "Alternative"))

    # 3. Colab用ダウンロードアーカイブ作成
    if files_to_download:
        print(f"\n{len(files_to_download)} ファイルをアーカイブ中...")

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archive_name = f"nerfstudio_colab_results_{timestamp}.zip"
        archive_path = os.path.join(download_dir, archive_name)

        # ファイルサイズ確認
        total_size = sum(os.path.getsize(file_path) for file_path, _ in files_to_download)
        print(f"アーカイブ対象サイズ: {total_size / (1024**2):.1f} MB")

        # Colabダウンロード制限チェック（500MB）
        colab_limit = 500 * 1024 * 1024
        if total_size > colab_limit:
            print(f"⚠️  サイズが大きいため、重要ファイルのみを選択します")
            # 重要度順にファイルを選択
            priority_types = ['Config', 'Checkpoint', 'Cameras', 'Transforms']
            selected_files = []
            current_size = 0

            for priority_type in priority_types:
                for file_path, file_type in files_to_download:
                    if file_type == priority_type:
                        file_size = os.path.getsize(file_path)
                        if current_size + file_size < colab_limit:
                            selected_files.append((file_path, file_type))
                            current_size += file_size

            files_to_download = selected_files
            print(f"選択されたファイル数: {len(files_to_download)}")
            print(f"最終サイズ: {current_size / (1024**2):.1f} MB")

        # ZIPアーカイブ作成
        with zipfile.ZipFile(archive_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_path, file_type in files_to_download:
                # Colab用のファイル名調整
                arcname = f"{file_type}/{os.path.basename(file_path)}"
                zipf.write(file_path, arcname)
                print(f"  ✓ {file_type}: {os.path.basename(file_path)}")

            # Colab環境情報を追加
            colab_info = {
                "creation_date": datetime.now().isoformat(),
                "environment": "Google Colab",
                "total_files": len(files_to_download),
                "archive_size_mb": os.path.getsize(archive_path) / (1024**2),
                "file_types": list(set(file_type for _, file_type in files_to_download)),
                "download_instructions": "Extract and use with local nerfstudio installation"
            }

            info_json = json.dumps(colab_info, indent=2)
            zipf.writestr("colab_export_info.json", info_json)

        # 4. Colab追加ファイルの準備
        additional_files = []

        # 可視化画像があれば追加
        viz_files = [
            "/content/drive/MyDrive/processed/colab_training_metrics.png",
            "/content/drive/MyDrive/processed/colab_render_samples.png"
        ]

        for viz_file in viz_files:
            if os.path.exists(viz_file):
                additional_files.append(viz_file)

        # 5. Colabダウンロード実行
        print(f"\n=== Google Colab ダウンロード開始 ===")

        try:
            # メインアーカイブをダウンロード
            archive_size_mb = os.path.getsize(archive_path) / (1024**2)
            print(f"メインアーカイブ: {archive_name} ({archive_size_mb:.1f} MB)")
            files.download(archive_path)
            print(f"✓ {archive_name} のダウンロードが完了しました")

            # 追加ファイルをダウンロード
            for additional_file in additional_files:
                try:
                    files.download(additional_file)
                    print(f"✓ {os.path.basename(additional_file)} のダウンロードが完了しました")
                except Exception as e:
                    print(f"⚠️  {os.path.basename(additional_file)} のダウンロードに失敗: {e}")

            print(f"\n=== ダウンロード完了 ===")
            print(f"ローカル環境での使用方法:")
            print(f"1. {archive_name} を解凍")
            print(f"2. 各フォルダの説明:")
            print(f"   - Config/: nerfstudio設定ファイル")
            print(f"   - Checkpoint/: 学習済みモデル")
            print(f"   - Cameras/: カメラパラメータ")
            print(f"   - Transforms/: 座標変換情報")
            print(f"   - PointCloud/: 3D点群データ")
            print(f"   - GaussianSplat/: Gaussian Splatting用")
            print(f"3. ローカルnerfstudioでの読み込み・レンダリング")

        except Exception as e:
            print(f"ダウンロードエラー: {e}")
            print(f"ファイルは {download_dir} に保存されています")

    else:
        print("ダウンロード可能なファイルが見つかりません")
        print("セル5での学習を完了してから実行してください")

else:
    print("このセルはGoogle Colab環境でのみ動作します")
    print("ローカル環境では手動でファイルをコピーしてください")

print(f"\nGoogle Colab ダウンロード処理が完了しました")